In [164]:
import os
import gzip
import io
import json
import pprint
from zipfile import ZipFile

# Path &File name sanitizer
from pathvalidate import sanitize_filename

---
### Note
Before running this script, install the "pathvalidate" module

see: https://pypi.org/project/pathvalidate/#installation-pip

run: pip install pathvalidate

---

In [165]:
# SET SCRIPT VARIABLES

myVars = {
    'paths': {
        'paprika_export': './',
        'paprika_jekyll': '../_recipes_md/'
    },
    'files': {
        'paprika_export': 'RecipeTest.paprikarecipes'
    }
}
# Learning to Pretty Print DICTs. I like to see the data I am sculpting with
#pp = pprint.PrettyPrinter(indent=4)
#pp.pprint(myVars)


In [ ]:
# If our export dir does not exist, create it
if not os.path.exists(myVars['paths']['paprika_jekyll']):
    os.mkdir(myVars['paths']['paprika_jekyll'])

In [166]:
# Function found on Gist 😸
# https://gist.github.com/Garrett-R/dc6f08fc1eab63f94d2cbb89cb61c33d
# This takes the GZIPed string from the files and retruns the unzipped JSON
def gunzip_bytes_obj(bytes_obj):
    in_ = io.BytesIO()
    in_.write(bytes_obj)
    in_.seek(0)
    with gzip.GzipFile(fileobj=in_, mode='rb') as fo:
        gunzipped_bytes_obj = fo.read()

    return gunzipped_bytes_obj.decode()

In [167]:
with ZipFile('RecipeTest.paprikarecipes', 'r') as zipObj:
    for filename in zipObj.namelist():
        #print('---------------------------------------')
        #print('OFile: ' + filename)
        
        with zipObj.open(filename) as f:
            data = f.read()
            
            # Using that function above to unGZIP the fie contents
            data_json = gunzip_bytes_obj(data)
 
            # "loads" that JSONinto a Python object
            # https://docs.python.org/3/library/json.html#basic-usage
            d = json.loads(data_json)
            
            # Prepare a cleaner filename we'll want for the Markdown Ouput (no spaces, etc)
            #
            # Get the "created on" date of the recpie in Paprika
            created = d["created"]
            
            # A few things going on here
            # 1. we slice the created date so as to only get the date, not the time. learn about slice: https://www.freecodecamp.org/news/how-to-substring-a-string-in-python/
            # 2. we run "sanitize_filename" (installed via PIP and imported above)
            # 3. we replace spaces with dashes
            fileName = sanitize_filename(created[0:10] + "-" + d["name"]).replace(" ","-") + ".md"
            mdFilePath = myVars['paths']['paprika_jekyll'] + fileName
            
            # Print out some stuff to see
            #print('RName: ' + d["name"])
            #print('FName: ' + fileName)
            #print('Notes: ' + d["notes"])
            #print('CTime: ' + d["cook_time"])
            #print('PTime: ' + d["prep_time"])
            #print('Ratng: ' + str(d["rating"]))
            #print('NFPath: ' + mdFilePath)

            # Create a string of Markdown
            # So this will require some "design." What do we want to include from the export?
            # How should it be styled? What do we jam into the metadata/frontmatter
            # What do we include as #tags in the body?
            
            output  = "---\n"
            output += "created: " + created + "\n"
            output += "---\n"
            output += "# " + d["name"] + "\n"
            if d["prep_time"] is not None:
                output += "## Prep time\n" + d["prep_time"] + "\n\n"
            if d["cook_time"] is not None:
                output += "## Cook time\n" + d["cook_time"] + "\n\n"
                
            if d["image_url"] is not None:
                output += "![]("+d["image_url"]+")\n\n"
                
            if d["description"] is not None:
                output += "## Description\n" + d["description"] + "\n\n---\n"
                
            if d["ingredients"] is not None:
                output += "## Ingredients\n" + d["ingredients"].replace("\n","<br>") + "\n\n---\n"
            if d["directions"] is not None:
                output += "## Directions\n" + d["directions"] + "\n\n---\n"

            if d["notes"] is not None:
                output += "## Notes\n" + d["notes"] + "\n\n---\n"

            
            #print(output)
            
            
            # Create/Open a text file for each recipe and write the above Markdown string into it
            f = open(mdFilePath, 'w')
            f.write(output)
            f.close()
            # Close the file
        
        

In [112]:
# comment out for now

#for filename in os.listdir('/Users/jito/GitHub/joi.github.io/_recipes'):
#    if filename.endswith(".asm") or filename.endswith(".py"): 
  #      print(os.path.join('/Users/jito/GitHub/joi.github.io/_recipes', filename))
  #      continue
 #   else:
 #       continue

In [103]:
#with open('RecipeTest.paprikarecipes', encoding='utf-8') as f:
#  data = json.load(f.decode("utf-8","ignore"))
#print (data)